# PaloBoost Experiments

This script analyzes the performance of PaloBoost (Gradient Boosting with Pruning and Adaptive Learning Rate using Out-of-Bag Samples) against to Scikit-learn GBM and XGBoost. 

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
from collections import defaultdict
import time

from bonsai.ensemble.paloboost import PaloBoost
from bonsai.ensemble.gbm import GBM

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

import preprocessing as pp

%matplotlib inline

In [ ]:
def get_reg_perf(models, X_train ,y_train, X_test, y_test):
    performance = {}
    for name, model in models.items():
        start = time.time()
        model.fit(X_train, y_train)
        time_fit = time.time() - start
        print("  {0}: {1:.5f} sec...".format(name, time_fit))
        performance[name] = []
        if "XGBoost" not in name:
            for i, y_hat_i in enumerate(model.staged_predict(X_test)):
                performance[name].append(np.clip(r2_score(y_test, y_hat_i), 0, 1))
        else:
            for i in range(n_estimators):
                y_hat_i = model.predict(X_test, ntree_limit=i+1)
                performance[name].append(np.clip(r2_score(y_test, y_hat_i), 0, 1))
    perf_df = pd.DataFrame(performance)
    perf_df.columns = ["{0} ({1:.5f}*)".format(c, v) 
                       for c, v in zip(perf_df.columns, perf_df.max())]
    return perf_df
    
def get_cls_perf(models, X_train, y_train, X_test, y_test):
    performance = {}
    for name, model in models.items():
        start = time.time()
        model.fit(X_train, y_train)
        time_fit = time.time() - start
        print("  {0}: {1:.5f} sec...".format(name, time_fit))
        performance[name] = []        
        if "XGBoost" not in name:
            for i, y_hat_i in enumerate(model.staged_predict_proba(X_test)):
                performance[name].append(roc_auc_score(y_test, y_hat_i[:,1]))
        else:
            for i in range(n_estimators):
                y_hat_i = model.predict_proba(X_test, ntree_limit=i+1)
                performance[name].append(roc_auc_score(y_test, y_hat_i[:,1]))
    perf_df = pd.DataFrame(performance)
    perf_df.columns = ["{0} ({1:.5f}*)".format(c, v) 
                       for c, v in zip(perf_df.columns, perf_df.max())]
    return perf_df

def plot_perf(perf_df, task, proj, ylim=None):
    ax = perf_df.plot(figsize=(6, 4), style=["-","--",":","-."], lw=2.5, ylim=ylim)
    if task == "reg":
        ax.set(xlabel="Prediction stage", ylabel="R-squared")
        fig = ax.get_figure()
        fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}.pdf".format(task, proj, subsample, learning_rate))
    else:
        ax.set(xlabel="Prediction stage", ylabel="AUROC")
        fig = ax.get_figure()
        fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}.pdf".format(task, proj, subsample, learning_rate))
        
def plot_prune_ratio(models, task, proj):
    paloboost = models["0. PaloBoost    "]
    prune_df = pd.DataFrame(paloboost.get_prune_stats())
    prune_ratio = (prune_df[1] - prune_df[2])/prune_df[1]
    ax = prune_ratio.plot(figsize=(6, 4), alpha=0.5)
    rolling = prune_ratio.rolling(window=20).mean().plot(ax=ax, lw=2.5, alpha=1)
    ax.set(xlabel="Prediction stage", ylabel="Prune Ratio")
    fig = ax.get_figure()
    fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}_prrt.pdf".format(task, proj, subsample, learning_rate))

def plot_avg_lr(models, task, proj):
    paloboost = models["0. PaloBoost    "]
    lr_df = pd.DataFrame(paloboost.get_lr_stats())
    ax = lr_df[1].plot(figsize=(6, 4), alpha=0.5)
    rolling = lr_df[1].rolling(window=20).mean().plot(ax=ax, lw=2.5, alpha=1)
    ax.set(xlabel="Prediction stage", ylabel="Average Learning Rate")
    fig = ax.get_figure()
    fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}_avglr.pdf".format(task, proj, subsample, learning_rate))

In [ ]:
# Parameters
n_estimators = 200
learning_rate = 1.0 # 1.0, 0.5, 0.1
test_size = 0.7  # 30% training, 70% test - to highlight the overfitting aspect of the models
subsample = 0.7
max_depth = 5

# Regression Task - Mercedes Challenge

We use the dataset from [one of the Kaggle competitions - Mercedes-Benz Greener Manufacturing](https://www.kaggle.com/c/mercedes-benz-greener-manufacturing). We minimally preprocess the data - for more information about how we preprocess the data, please read `preprocess.py` in the same folder.

In [ ]:
data = pd.read_csv("../data/kaggle-mercedes/train.csv")
col_names = data.columns
col_names_x = [cname for cname in col_names if cname not in ["ID", "y"]]
X = pp.simple_pp(data[col_names_x]).values
y = data["y"].values
print("- Avg(y): {}, Std(y): {}".format(np.mean(y), np.std(y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

The list of our benchmark models is as follows:
- `paloboost`: PaloBoost (GBM with Pruning and Adaptive Learning Rate)
- `gbm`: GBM implemented with Bonsai decision trees
- `xgboost`: XGBoost with Scikit-earn interface
- `sklearn`: Scikit Learn GBM implementation

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="gaussian",
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
        "1. SGTB-Bonsai": GBM(distribution="gaussian",
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
        "2. XGBoost      ": XGBRegressor(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample),
        "3. Scikit-Learn ": GradientBoostingRegressor(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample,
                    random_state=1)}

In [ ]:
perf_df = get_reg_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "reg", "mercedes", ylim=(0.0, 0.57))

In [ ]:
plot_prune_ratio(models, "reg", "mercedes")

In [ ]:
plot_avg_lr(models, "reg", "mercedes")

# Regression Task - Communities Crimes

Dataset from UCI: [https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime)

In [ ]:
data = pd.read_csv("../data/uci-communities/communities.txt", header=None, na_values="?")
col_names = data.columns
col_names_x = col_names[:-1]
col_name_y = col_names[-1]
X = pp.simple_pp(data[col_names_x]).values
y = data[col_name_y].values
print("- Avg(y): {}, Std(y): {}".format(np.mean(y), np.std(y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="gaussian",
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
        "1. SGTB-Bonsai": GBM(distribution="gaussian",
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
        "2. XGBoost      ": XGBRegressor(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample)}

In [ ]:
perf_df = get_reg_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "reg", "communities", ylim=(0.0, 0.68))

In [ ]:
plot_prune_ratio(models, "reg", "communities")

In [ ]:
plot_avg_lr(models, "reg", "communities")

# Classification Task - Amazon Access Challenge

We use the dataset from [one of the Kaggle competitions - Amazon.com Employee Access Challenge](https://www.kaggle.com/c/amazon-employee-access-challenge). We minimally preprocess the data - for more information about how we preprocess the data, please read `preprocess.py` in the same folder.

In [ ]:
data = pd.read_csv("../data/kaggle-amazon/train.csv")
col_names_x = list(set(data.columns)-set(["ACTION"]))
for cname in col_names_x:
    data[cname] = data[cname].astype("object")
X = pp.simple_pp(data[col_names_x]).values
y = data["ACTION"].values
print("- Avg(y): {}".format(np.mean(y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "1. SGTB-Bonsai": GBM(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "2. XGBoost      ": XGBClassifier(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample),
         "3. Scikit-Learn ": GradientBoostingClassifier(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample,
                    random_state=1)}

In [ ]:
perf_df = get_cls_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "cls", "amazon", ylim=(0.58, 0.75))

In [ ]:
plot_prune_ratio(models, "cls", "amazon")

In [ ]:
plot_avg_lr(models, "cls", "amazon")

# Classification Task - Pulsar Detection

Pulsar detection. UCI dataset from [https://archive.ics.uci.edu/ml/datasets/HTRU2](https://archive.ics.uci.edu/ml/datasets/HTRU2)

In [ ]:
data = pd.read_csv("../data/uci-htru/HTRU_2.csv", header=None)
col_names = data.columns
col_names_x = col_names[:-1]
col_name_y = col_names[-1]
X = pp.simple_pp(data[col_names_x]).values
y = data[col_name_y].values
print("- Avg(y): {}".format(np.mean(y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "1. SGTB-Bonsai": GBM(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "2. XGBoost      ": XGBClassifier(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample),
         "3. Scikit-Learn": GradientBoostingClassifier(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample,
                    random_state=1)}

In [ ]:
perf_df = get_cls_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "cls", "htru", ylim=(0.85, 0.99))

In [ ]:
plot_prune_ratio(models, "cls", "htru")

In [ ]:
plot_avg_lr(models, "cls", "htru")

# Classification Task - Carvana Don't Get Kicked! Challenge

We use the dataset from [one of the Kaggle competitions - Carvana Don't Get Kicked! Challenge](https://www.kaggle.com/c/DontGetKicked). We minimally preprocess the data - for more information about how we preprocess the data, please read `preprocess.py` in the same folder.

In [ ]:
data = pd.read_csv("../data/kaggle-carvana/training.csv")
y = data["IsBadBuy"].values
data = data.drop(["RefId", "IsBadBuy", "PurchDate"], axis=1)
X = pp.simple_pp(data).values
print("- Avg(y): {}".format(np.mean(y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "1. SGTB-Bonsai": GBM(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "2. XGBoost      ": XGBClassifier(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample)}

In [ ]:
perf_df = get_cls_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "cls", "carvana", ylim=(0.65, 0.78))

In [ ]:
plot_prune_ratio(models, "cls", "carvana")

In [ ]:
plot_avg_lr(models, "cls", "carvana")

# Classification Task - BNP Paribas Challenge

We use the dataset from [one of the Kaggle competitions - BNP Paribas Cardif Claims Management](https://www.kaggle.com/c/bnp-paribas-cardif-claims-management). We minimally preprocess the data - for more information about how we preprocess the data, please read `preprocess.py` in the same folder.

In [ ]:
data = pd.read_csv("../data/kaggle-bnp/train.csv")
y = data["target"].values
data = data.drop(["ID", "target"], axis=1)
X = pp.simple_pp(data).values
print("- Avg(y): {}".format(np.mean(y)))
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "1. SGTB-Bonsai": GBM(distribution="bernoulli",
                        n_estimators=n_estimators, 
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
         "2. XGBoost      ": XGBClassifier(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample)}

In [ ]:
perf_df = get_cls_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "cls", "bnp", ylim=(0.64, 0.76))

In [ ]:
plot_prune_ratio(models, "cls", "bnp")

In [ ]:
plot_avg_lr(models, "cls", "bnp")

# Regression Task - Friedman Simulated Data

[http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html)

In [ ]:
from sklearn.datasets import make_friedman1
X, y = make_friedman1(n_samples=10000, noise=5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size = test_size,
                                            random_state=1)

In [ ]:
models = {"0. PaloBoost    ": PaloBoost(distribution="gaussian",
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
        "1. SGTB-Bonsai": GBM(distribution="gaussian",
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        max_depth=max_depth, 
                        subsample=subsample),
        "2. XGBoost      ": XGBRegressor(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample),
        "3. Scikit-Learn ": GradientBoostingRegressor(
                    n_estimators=n_estimators, 
                    learning_rate=learning_rate,
                    max_depth=max_depth, 
                    subsample=subsample,
                    random_state=1)}

In [ ]:
perf_df = get_reg_perf(models, X_train, y_train, X_test, y_test)
plot_perf(perf_df, "reg", "friedman", ylim=(0.0,0.5))

In [ ]:
plot_prune_ratio(models, "reg", "friedman")

In [ ]:
plot_avg_lr(models, "reg", "friedman")

In [ ]:
fi = {"colname": ["x{}".format(i) for i in np.arange(X.shape[1])]}
for name, model in models.items():
    fi[name] = model.feature_importances_
fi = pd.DataFrame(fi)
fi = fi.set_index("colname")
ax = fi.plot.bar(subplots=True, figsize=(14, 4), layout=(1,4), legend=False)[0][0]
fig = ax.get_figure()
fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}_fi.pdf".format("reg", "friedman", subsample, learning_rate))

In [ ]:
fi_staged = defaultdict(list)
for stage, fi in enumerate(models["0. PaloBoost    "].get_staged_feature_importances()):
    fi_staged["stage"].append(stage)
    for j, fi_j in enumerate(fi):
        fi_staged["x{}".format(j)].append(fi_j)
fi_staged = pd.DataFrame(fi_staged)
fi_staged = fi_staged.set_index("stage")
ax = fi_staged.plot()
ax.set(xlabel="Prediction stage", ylabel="Feature importance")
fig = ax.get_figure()
fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}_fi_staged0.pdf".format("reg", "friedman", subsample, learning_rate))

In [ ]:
fi_staged = defaultdict(list)
for stage, fi in enumerate(models["1. SGTB-Bonsai"].get_staged_feature_importances()):
    fi_staged["stage"].append(stage)
    for j, fi_j in enumerate(fi):
        fi_staged["x{}".format(j)].append(fi_j)
fi_staged = pd.DataFrame(fi_staged)
fi_staged = fi_staged.set_index("stage")
ax = fi_staged.plot()
ax.set(xlabel="Prediction stage", ylabel="Feature importance")
fig = ax.get_figure()
fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}_fi_staged1.pdf".format("reg", "friedman", subsample, learning_rate))

In [ ]:
# for speed benchmark
if False: 
    n_list = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]
    speed_df = defaultdict(list)
    for n_sample in n_list:
        X, y = make_friedman1(n_samples=n_sample, noise=5, random_state=0)
        speed_df["n_sample"].append(n_sample)
        for name, model in models.items():
            start_time = time.time()
            model.fit(X, y)
            fit_time = time.time() - start_time
            speed_df[name].append(fit_time)
            print(name, n_sample, fit_time)
    speed_df = pd.DataFrame(speed_df)
    speed_df = speed_df.set_index("n_sample")
    ax = speed_df.plot(style=[".-",".--",".:",".-."], logx=True, logy=True, figsize=(6, 4))
    ax.set(xlabel="Number of Samples", ylabel="Training Time (in Seconds)")
    fig = ax.get_figure()
    fig.savefig("figures-paloboost/{}_{}_ss{}_lr{}_speed.pdf".format("reg", "friedman", subsample, learning_rate))

In [ ]:
np.std(y)